In [13]:
import pandas as pd
import numpy as np
from copy import deepcopy
from random import randint 
import time
import random 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, plot_confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import roc_auc_score
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from sklearn import metrics

In [14]:
#from orion.contrib.envs import load_env
#load_env()
import boto3
#from orion.sources import S3Source
aws_bucket = 'kilimanjaro-prod-datalake'
s3 = boto3.client('s3')

In [15]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [16]:
object_ = 'masters/datascience/emma/retail.csv'
retail = s3.get_object(Bucket=aws_bucket, Key=object_)
retail = pd.read_csv(retail['Body'], index_col=0)

In [17]:
retail = retail.rename(columns={'class':'cluster'})
retail.head(1)

,ASP,cluster,loyaltyaccount_No,loyaltyaccount_Yes,gender_female,gender_male,gender_unknown,shipcountry_Albania,shipcountry_Armenia,shipcountry_Australia,...,category_Childrens,category_Infant,category_Junior,category_Mens,category_Miscellaneous,category_Nursery,category_Womens,divisioncode_ACCESSORY,divisioncode_APPAREL,divisioncode_FOOTWEAR
0,59.92,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [18]:
wine_white = pd.read_csv('winequality-white.csv', delimiter=';')
wine_red = pd.read_csv('winequality-red.csv', delimiter=';')

In [140]:
# Red Wine Quality Low vs High
WineRLvH = deepcopy(wine_red)
WineRLvH = WineRLvH[(WineRLvH['quality'] <5)| (WineRLvH['quality'] >6)]
WineRLvH.loc[WineRLvH.quality >= 7, "class"] = 0
WineRLvH.loc[WineRLvH.quality <= 4, "class"] = 1
WineRLvH['class'] = WineRLvH['class'].astype("int")
WineRLvH['class'] = WineRLvH['class'].astype("category")
WineRLvH = WineRLvH.drop(columns=['quality'])
print(WineRLvH['class'].value_counts())
WineRLvH.name ='D12 - Wine_red_LvH'

0    217
1     63
Name: class, dtype: int64


In [279]:
# White Wine Quality Low vs High
WineWLvH = deepcopy(wine_white)
WineWLvH = WineWLvH[(WineWLvH['quality'] <5)| (WineWLvH['quality'] >6)]
WineWLvH.loc[WineWLvH.quality >= 7, "class"] = 0
WineWLvH.loc[WineWLvH.quality <= 4, "class"] = 1
WineWLvH['class'] = WineWLvH['class'].astype("int")
WineWLvH['class'] = WineWLvH['class'].astype("category")
WineWLvH = WineWLvH.drop(columns=['quality'])
print(WineWLvH['class'].value_counts())
WineWLvH.name = 'D10 - Wine_white_LvH'

0    1060
1     183
Name: class, dtype: int64


In [10]:
# Cust Summary focusing on singular minority
cust_sum2 = deepcopy(retail)
cust_sum2.loc[cust_sum2.cluster ==0, "class"] = 0
cust_sum2.loc[cust_sum2.cluster >= 1, "class"] = 1
cust_sum2['class'] = cust_sum2['class'].astype("int")
cust_sum2['class'] = cust_sum2['class'].astype("category")
cust_sum2 = cust_sum2.drop(columns=['cluster'])
print(cust_sum2['class'].value_counts())
cust_sum2.name ='Retail Data -2'

0    1935848
1     269025
Name: class, dtype: int64


In [293]:
# train test/ scaling
def data_prep (data, seed):
  X= data.drop('class',axis=1).copy()
  y = data['class'].copy()
  y = y.astype('category')
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed, shuffle=y, stratify=y) # add ssed

  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train_scaled = scaler.transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
  X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

  return X_train_scaled, X_test_scaled, y_train, y_test

In [320]:
X_train_scaled, X_test_scaled, y_train, y_test = data_prep(cust_sum2, 3)

In [321]:
len(X_train_scaled)

1653654

In [322]:
X_train_scaled

,ASP,loyaltyaccount_No,loyaltyaccount_Yes,gender_female,gender_male,gender_unknown,shipcountry_Albania,shipcountry_Armenia,shipcountry_Australia,shipcountry_Austria,...,category_Childrens,category_Infant,category_Junior,category_Mens,category_Miscellaneous,category_Nursery,category_Womens,divisioncode_ACCESSORY,divisioncode_APPAREL,divisioncode_FOOTWEAR
0,0.426793,-1.187911,1.187911,0.992984,-0.917115,-0.203148,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,-0.405054,-1.208662,-0.266557,-0.114176,2.518249,-0.30401,1.415001,-1.180731
1,-0.997094,0.841814,-0.841814,-1.007065,1.090376,-0.203148,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,-0.405054,0.827361,-0.266557,-0.114176,-0.397101,-0.30401,1.415001,-1.180731
2,-1.007324,0.841814,-0.841814,0.992984,-0.917115,-0.203148,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,-0.405054,0.827361,-0.266557,-0.114176,-0.397101,-0.30401,1.415001,-1.180731
3,-0.203488,0.841814,-0.841814,-1.007065,-0.917115,4.922521,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,2.468806,-1.208662,-0.266557,-0.114176,-0.397101,-0.30401,-0.706713,0.846933
4,1.865296,0.841814,-0.841814,0.992984,-0.917115,-0.203148,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,-0.405054,-1.208662,-0.266557,-0.114176,2.518249,-0.30401,-0.706713,0.846933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1653649,0.334352,-1.187911,1.187911,-1.007065,1.090376,-0.203148,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,-0.405054,0.827361,-0.266557,-0.114176,-0.397101,-0.30401,1.415001,-1.180731
1653650,0.436659,0.841814,-0.841814,0.992984,-0.917115,-0.203148,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,2.468806,-1.208662,-0.266557,-0.114176,-0.397101,-0.30401,-0.706713,0.846933
1653651,-0.872134,0.841814,-0.841814,-1.007065,-0.917115,4.922521,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,2.468806,-1.208662,-0.266557,-0.114176,-0.397101,-0.30401,-0.706713,0.846933
1653652,0.517773,-1.187911,1.187911,0.992984,-0.917115,-0.203148,-0.004979,-0.002199,-0.040373,-0.009525,...,-0.043028,-0.118682,-0.405054,0.827361,-0.266557,-0.114176,-0.397101,-0.30401,1.415001,-1.180731


In [323]:
y_train = y_train.reset_index(drop=True)
X_train_scaled = X_train_scaled.reset_index(drop=True)
X_train_scaled['class'] = y_train

In [324]:
label_f = np.array(y_train[y_train==0])
label_t = np.array(y_train[y_train==1])

In [325]:
data_f = np.array(X_train_scaled[y_train==0])
data_t = np.array(X_train_scaled[y_train==1])

In [326]:
len(data_t)

201769

In [327]:
len(data_f)

1451885

In [328]:
data_t.shape

(201769, 85)

In [356]:
x_mu = data_t - np.mean(data_t)

In [362]:
x_mu.shape

(201769, 85)

In [336]:
data_t = data_t+0.00001*np.random.rand((data_t.shape)[0],(data_t.shape)[1])

In [357]:
cov = np.cov(data_t.T)

In [358]:
inv_covmat = np.linalg.inv(cov)

In [359]:
inv_covmat

array([[ 2.19548517e+00,  6.68392637e+02,  6.68341503e+02, ...,
         8.33124403e+02,  8.71166753e+02,  5.28437901e+02],
       [ 6.68393495e+02,  5.99975894e+10,  5.99975892e+10, ...,
        -9.66306195e+07, -1.01113484e+08,  3.17566046e+08],
       [ 6.68342361e+02,  5.99975892e+10,  5.99975889e+10, ...,
        -9.66298060e+07, -1.01112632e+08,  3.17567431e+08],
       ...,
       [ 8.33117754e+02, -9.66491986e+07, -9.66483851e+07, ...,
         4.91787252e+10,  5.14598092e+10, -2.11683732e+08],
       [ 8.71159813e+02, -1.01133132e+08, -1.01132281e+08, ...,
         5.14598092e+10,  5.38466980e+10, -2.21501021e+08],
       [ 5.28437886e+02,  3.17566440e+08,  3.17567825e+08, ...,
        -2.11686120e+08, -2.21503520e+08,  1.19876242e+11]])

In [340]:
left = np.dot(x_mu, inv_covmat)
mahal = np.dot(left, x_mu.T)
len(mahal.diagonal())

MemoryError: Unable to allocate 303. GiB for an array with shape (201769, 201769) and data type float64

In [ ]:
def mahalanobis(x=None, data=None, cov=None):

    x_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = np.linalg.inv(cov)
    left = np.dot(x_mu, inv_covmat)
    mahal = np.dot(left, x_mu.T)
    return mahal.diagonal()

In [2]:
import numpy as np
import pandas as pd 
import scipy as stats

data = {'score': [91, 93, 72, 87, 86, 73, 68, 87, 78, 99, 95, 76, 84, 96, 76, 80, 83, 84, 73, 74],
        'hours': [16, 6, 3, 1, 2, 3, 2, 5, 2, 5, 2, 3, 4, 3, 3, 3, 4, 3, 4, 4],
        'prep': [3, 4, 0, 3, 4, 0, 1, 2, 1, 2, 3, 3, 3, 2, 2, 2, 3, 3, 2, 2],
        'grade': [70, 88, 80, 83, 88, 84, 78, 94, 90, 93, 89, 82, 95, 94, 81, 93, 93, 90, 89, 89]
        }

df = pd.DataFrame(data,columns=['score', 'hours', 'prep','grade'])
df.head()

,score,hours,prep,grade
0,91,16,3,70
1,93,6,4,88
2,72,3,0,80
3,87,1,3,83
4,86,2,4,88


In [6]:
x=df 
data=df[['score', 'hours', 'prep', 'grade']]

In [7]:
x_mu = x - np.mean(data)

In [8]:
x_mu.shape

(20, 4)

In [11]:
cov = np.cov(df.values.T)
inv_covmat = np.linalg.inv(cov)
left = np.dot(x_mu, inv_covmat)
mahal = np.dot(left, x_mu.T).diagonal()

In [12]:
mahal

array([16.50196304,  2.63928638,  4.8507973 ,  5.20126123,  3.82873415,
        4.09056333,  4.28363027,  2.4198736 ,  1.65195755,  5.65782528,
        3.96587697,  2.93501779,  2.81021087,  4.3682945 ,  1.56101651,
        1.45950688,  2.02457478,  0.75025359,  2.73512915,  2.26422684])

In [370]:
cov.shape

(4, 4)

In [371]:
left.shape

(20, 4)

In [372]:
mahal.shape

(20, 20)